In [9]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()
spark

In [10]:
from pyspark import SparkContext
 
sc = SparkContext.getOrCreate()

In [11]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv("C:/Users/prepradi/Downloads/Health Care_Patient_survey_source.csv")

In [12]:

df.printSchema()
df.show(5)


root
 |-- Provider ID: double (nullable = true)
 |-- Hospital Name: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZIP Code: integer (nullable = true)
 |-- County Name: string (nullable = true)
 |-- Phone Number: long (nullable = true)
 |-- Measure ID: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Answer Description: string (nullable = true)
 |-- Patient Survey Star Rating: string (nullable = true)
 |-- Patient Survey Star Rating Footnote: string (nullable = true)
 |-- Answer Percent: string (nullable = true)
 |-- Answer Percent Footnote: string (nullable = true)
 |-- Linear Mean Value: string (nullable = true)
 |-- Number of Completed Surveys: string (nullable = true)
 |-- Number of Completed Surveys Footnote: string (nullable = true)
 |-- Survey Response Rate Percent: string (nullable = true)
 |-- Survey Response Rate Percent Footnote: string (nullable = true)
 |-- M

In [15]:
#3.	Top 3 Countries which has the highest survey rate
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg


spark = SparkSession.builder \
    .appName("TopSurveyStates") \
    .getOrCreate()


file_path = "Health Care_Patient_survey_source 3.csv"
df = spark.read.option("header", "true").option("inferSchema", "true").csv("C:/Users/prepradi/Downloads/Health Care_Patient_survey_source.csv")


df = df.select([col(c.strip()).alias(c.strip()) for c in df.columns])


df = df.withColumn("Survey Response Rate Percent", col("Survey Response Rate Percent").cast("double"))


avg_response = df.groupBy("State") \
    .agg(avg("Survey Response Rate Percent").alias("Average_Survey_Response_Rate")) \
    .orderBy(col("Average_Survey_Response_Rate").desc())


avg_response.show(3)


+-----+----------------------------+
|State|Average_Survey_Response_Rate|
+-----+----------------------------+
|   NE|           37.70422535211268|
|   WI|          35.235772357723576|
|   MN|           34.26126126126126|
+-----+----------------------------+
only showing top 3 rows



In [16]:
#List the details of the patients who has completed the survey on the same date.
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

# Initialize Spark session
spark = SparkSession.builder.appName("SurveySameDate").getOrCreate()

# Load the CSV file
file_path = "Health Care_Patient_survey_source 3.csv"
df = spark.read.option("header", "true").option("inferSchema", "true").csv("C:/Users/prepradi/Downloads/Health Care_Patient_survey_source.csv")

# Clean column names
df = df.select([col(c.strip()).alias(c.strip()) for c in df.columns])

# Group by 'Measure End Date' and count entries
date_counts = df.groupBy("Measure End Date").agg(count("*").alias("count"))

# Filter for dates with more than one entry
duplicate_dates = date_counts.filter(col("count") > 1)

# Join back to original DataFrame to get full details
result_df = df.join(duplicate_dates, on="Measure End Date", how="inner")

# Show the result
result_df.show(10, truncate=False)


+----------------+-----------+--------------------------------+----------------------+------+-----+--------+-----------+------------+---------------------+------------------------------------------------------------------------------------------+-----------------------------------------------------+--------------------------+-----------------------------------+--------------+-----------------------+-----------------+---------------------------+------------------------------------+----------------------------+-------------------------------------+------------------+---------------------------------------------------------------+------+
|Measure End Date|Provider ID|Hospital Name                   |Address               |City  |State|ZIP Code|County Name|Phone Number|Measure ID           |Question                                                                                  |Answer Description                                   |Patient Survey Star Rating|Patient Survey Star Rating Foo

In [18]:
#Top 10 hospitals based on Survey Response Rating 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg


spark = SparkSession.builder.appName("TopHospitalsBySurveyRating").getOrCreate()


file_path = "Health Care_Patient_survey_source 3.csv"
df = spark.read.option("header", "true").option("inferSchema", "true").csv("C:/Users/prepradi/Downloads/Health Care_Patient_survey_source.csv")


df = df.select([col(c.strip()).alias(c.strip()) for c in df.columns])


df = df.withColumn("Survey Response Rate Percent", col("Survey Response Rate Percent").cast("double"))


avg_response = df.groupBy("Hospital Name") \
    .agg(avg("Survey Response Rate Percent").alias("Average_Survey_Response_Rate")) \
    .orderBy(col("Average_Survey_Response_Rate").desc())


avg_response.show(10, truncate=False)


+-----------------------------------+----------------------------+
|Hospital Name                      |Average_Survey_Response_Rate|
+-----------------------------------+----------------------------+
|REGIONAL GENERAL HOSPITAL WILLISTON|82.0                        |
|OAKLEAF SURGICAL HOSPITAL          |71.0                        |
|SALINA SURGICAL HOSPITAL           |69.0                        |
|MIDWEST SURGICAL HOSPITAL LLC      |69.0                        |
|PHYSICIAN'S CARE SURGICAL HOSPITAL |69.0                        |
|KANSAS SURGERY & RECOVERY CENTER   |67.0                        |
|SURGICAL INSTITUTE OF READING      |67.0                        |
|NEBRASKA ORTHOPAEDIC HOSPITAL      |66.0                        |
|SURGICAL HOSPITAL AT SOUTHWOODS    |66.0                        |
|LINCOLN SURGICAL HOSPITAL          |64.0                        |
+-----------------------------------+----------------------------+
only showing top 10 rows

